In [49]:
import numpy as np
import cv2 as cv
from collections import deque

In [50]:
def count_opencv(img):
    n, labels = cv.connectedComponents(img.astype('uint8'), connectivity=4)

    return n - 1


def count_dfs(img):
    """Depth-wise search (breadth-wise search could be used as well)"""
    img = np.pad(img, pad_width=1)
    visited_mask = np.zeros(img.shape, dtype=bool)
    
    def search(y, x):
        stack = deque()
        stack.append((y, x))
        
        while len(stack) > 0:
            y, x = stack.pop()
            visited_mask[y, x] = True
            if img[y - 1, x] > 0 and not visited_mask[y - 1, x]:
                stack.append((y - 1, x))
            if img[y + 1, x] > 0 and not visited_mask[y + 1, x]:
                stack.append((y + 1, x))
            if img[y, x - 1] > 0 and not visited_mask[y, x - 1]:
                stack.append((y, x - 1))
            if img[y, x + 1] > 0 and not visited_mask[y, x + 1]:
                stack.append((y, x + 1))

    count = 0
    for i in range(1, img.shape[0] - 1):
        for j in range(1, img.shape[1] - 1):
            if img[i, j] == 0 or visited_mask[i, j]:
                continue
            
            count += 1
            search(i, j)
            
    return count

In [75]:
img1 = np.asarray([
    [0, 1, 0],
    [0, 0, 0],
    [0, 1, 1],
])
img2 = np.asarray([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
])
img3 = np.asarray([
    [0, 0, 0, 1],
    [0, 0, 1, 1],
    [0, 1, 0, 1],
])
np.random.seed(42)
img_list = np.random.randint(low=0, high=2, size=(100, 100, 100))

In [52]:
# Check open-cv solution
count_opencv(img1), \
    count_opencv(img2), \
    count_opencv(img3)

(2, 3, 2)

In [53]:
# Check custom solution
count_dfs(img1), \
    count_dfs(img2), \
    count_dfs(img3)

(2, 3, 2)

In [76]:
# Verify custom solution validity on the number of random samples
len(img_list) == sum([count_opencv(img) == count_dfs(img) for img in img_list])

True

In [39]:
%timeit -n 100_000 -r 10 count_opencv(img1)

876 ns ± 12.9 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)


In [79]:
%timeit -n 100_000 -r 10 count_dfs(img1)

11.5 μs ± 81.4 ns per loop (mean ± std. dev. of 10 runs, 100,000 loops each)


Evidence: custom solution is correct, but it takes 11 times more time to open-cv solution